# Calculate Prime Numbers and Plot Them

### Authors: 
- Christian Michelsen (Niels Bohr Institute)
- Troels C. Petersen (Niels Bohr Institute)

### Date:    
- 24-09-2018 (latest update)
- 06-10-2015 (originally)

***
Script calculating all prime numbers up to 10000 and plots them in a histogram, effectively counting the number of primes in intervals of 100.
***

First, import the relevant packages:

In [1]:
import numpy as np               
import matplotlib.pyplot as plt
from iminuit import Minuit
from probfit import Chi2Regression #, BinnedChi2, UnbinnedLH, BinnedLH
import sys                                         # Modules to see files and folders in directories


In [2]:
sys.path.append('../../External_Functions')
from ExternalFunctions import nice_string_output, add_text_to_ax # useful functions to print fit results on figure

Decide whether or not the program should save the plot along with the maxmimum prime number and the starting point:

In [ ]:
save_plots = True
Nmax = 10000              # Maximum prime number
N    = 2                  # Starting value

## Main program:


Calculating prime numbers comes down to taking a number (here `N`), and testing if it is prime by dividing it by all numbers up the the square root of it. If no number divides `N` (tested using the `%` operator, which returns the remainder: `6%3 = 0`, `5%2 = 1`, etc.), then `N` is prime!
***
See if you can follow this in python program form below. There are only 9 lines in total! The central part consists of a While-loop, which loops as long as the statement is true. It is true to begin with (`N=2`, `Nmax = 10000`), but at some point `N` will have increased enough to equal `Nmax`, and the looping stops.
***
We start by creating empty list for primes. We dont create a numpy array since we don't know the size of it before creation:

In [ ]:
primes = []

while (N < Nmax):
    # print("Potential Prime number I want to test: ", N)
    hit = False                # Define a variable "hit", with starting value "False".

    for i in range (2, int(np.sqrt(N)) + 1) :   # Loop over the possible divisors.
        # print("  I'm testing the first number with the second and get the third: ", N, i, N%i)

        if N%i == 0:           # If the remainder of the integer division is 0
            hit = True         # hit = "True" (i.e. the number being tested had a divisor, and is not a prime)
            break              # Skip the rest of the possible divisors.

    if not hit:             # If no number gave a perfect integer division (i.e. with remainder 0)...
        primes.append(N)
        print("I got a prime", N)   # ...the it is a prime number.

    N += 1                     # Increase N by one.

We can now convert the Python list into a Numpy array:

In [ ]:
primes = np.array(primes)
print("First 10 primes, ", primes[:10]) # prints the first 10 primes

We can check that there are no even primes except 2:

In [ ]:
print("  The even prime numbers are listed here: ", primes[(primes % 2) == 0])

## Draw output:

Having computed the prime numbers, we now try to plot the distribution of them to see if there are any interesting patterns. 

We first define the number of bins and the minimum/maximum x value:

In [ ]:
Nbins = 100
xmin = 0
xmax = Nmax
binwidth = int((xmax-xmin)/Nbins)

In this case we want the distribution of prime numbers as graph with error bars and not a regular histogram.  
Therefore we don't do `ax.hist(primes, bins=Nbins, ...)`, but rather bin it using Numpy:

In [ ]:
y, bin_edges = np.histogram(primes, bins=Nbins, range=(xmin, xmax))
x = bin_edges[:-1] + np.diff(bin_edges)       # Calculate the x-values as the center of the bins.
sy = np.sqrt(y)                                  # Assume Poissonian errors (more on that later!)

And then plot it:

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.errorbar(x, y, sy, fmt='.', label='Prime number distribution')
ax.set(xlim=(xmin, xmax), 
       xlabel="Prime Number", 
       ylabel=f"Frequency /{binwidth:4d}",    # To put the bin width in the y-axis label 
       title="Prime number density distribution");

## Fitting the data:

The __[Prime Number Theorem](http://en.wikipedia.org/wiki/Prime_number_theorem)__ states, that the number of prime numbers follows roughly the function $f(x) = \frac{1}{\ln(x)}$. We have binned our prime numbers, so we must have a function with a few more degrees of freedom: $f(x) = c_0 + \frac{c_1}{\ln(x)}$, where $c_0$ and $c_1$ are constants that needs to be fitted.

We define the function to fit:

In [ ]:
def fit_prime(x, c0, c1) :
    return c0 + c1 / np.log(x)

Make a chi2-regression object based on the fit function `fit_prime` and the `x`, `y` and `sy` values: 

In [ ]:
chi2_object = Chi2Regression(fit_prime, x, y, sy)
minuit = Minuit(chi2_object, pedantic=False, c0=0, c1=10) # Initializes the fit with given initial values
minuit.migrad(); # Fit the function to the data

Extract the fit parameters:

In [ ]:
c0, c1 = minuit.args                    # Get the output arguments
for name in minuit.parameters:          # Print the output (parameters of the fit)
    print(f"Fit value: {name} = {minuit.values[name]:.5f} +/- {minuit.errors[name]:.5f}")

And plot the fit along with some extra text to the plot:

In [ ]:
x_fit = np.linspace(xmin, xmax, 1000)[1:]   # Create the x-axis for the plot of the fitted function - Not from x[0] since x[0] is 0, which is not defined for the log!
y_fit = fit_prime(x_fit, c0, c1)    

ax.plot(x_fit, y_fit, '-', label='Prime number fit result')

ax.text(0.5, 0.6, r'Fit function: f(x) = $C_0 + \frac{C_1}{\log (x)}$', transform = ax.transAxes, fontdict={'size': 18}); # transform so relative coordinates
ax.legend(loc='best')

fig

We also want to extract the $\chi^2$ value (see `IntroToPlottingAndFitting.ipynb` if the command below is unclear):

In [ ]:
chi2_val = minuit.fval

And the number of degrees of freedom:

In [ ]:
N_NotEmptyBin = len(y > 0)
Ndof = N_NotEmptyBin - len(minuit.args)            # Number of degrees-of-freedom

We calculate the $\chi^2$ probability:

In [ ]:
from scipy import stats
chi2_prob =  stats.chi2.sf(chi2_val, Ndof)

And add the fit information to the plot:

In [ ]:
d = {'Entries':  N_NotEmptyBin,
     'Chi2':     chi2_val,
     'ndf':      Ndof,
     'Prob':     chi2_prob,
     'c0':       [minuit.values['c0'], minuit.errors['c0']],
     'c1':       [minuit.values['c1'], minuit.errors['c1']]
    }

text = nice_string_output(d, extra_spacing=-2, decimals=2)
add_text_to_ax(0.02, 0.95, text, ax, fontsize=12)
fig.tight_layout()
fig

Finally we save the plots if save_plots is True:

In [ ]:
if save_plots:
    fig.savefig('PrimeNumberDistribution.pdf', dpi=600)